
# Data Preprocessing
In this notebook we load and process the raw data to develop the final dataset for the IBM-Z Datathon. We make use of two main datasets for a list of all observed geoeffective CMEs from the post-SOHO era between 1996-2024, and two data sets for features and targets:

#### Geo-effective CMEs:
- The [Richardson and Cane list](https://izw1.caltech.edu/ACE/ASC/DATA/level3/icmetable2.htm); a list of near-Earth CMEs from 1996-2024.
- The [George Mason University CME/ICME list](http://solar.gmu.edu/heliophysics/index.php/GMU_CME/ICME_List); a list of geoeffective CMEs from 2007-2017.

#### Features and Targets:
- The [SOHO-LASCO CME Catalogue](https://cdaw.gsfc.nasa.gov/CME_list/); a list of all CMEs observed from 1996-2024 containing information on physical quantities.
- [OMNIWeb Plus data](https://omniweb.gsfc.nasa.gov/); a list of features associated with the solar wind and sunspot numbers.


## Cleaning the data:

In [456]:
# Importing libraries:
# For data manipulation
import pandas as pd

#For data visualisation:
import matplotlib.pyplot as plt
from statsmodels.formula.api import mixedlm


#### SOHO-LASCO Catalogue

We begin by loading in the SOHO-LASCO Catalogue to obtain the physical quantities for all CMEs. The original dataset had 11 total features. Most of the data was missing for the mass and kinetic energy hence these have been excluded. We have also excluded the second-order speeds as these are correlated with the linear speed. As a result this dataset contains the dates and times for each CME, together with five features:
- Central Position Angle in degrees.
- Angular Width in degrees.
- Linear Speed in km/s.
- Acceleration in km/s$^2$.
- Measurement Position Angle in degrees.


In [457]:
# Adding filepaths as variables
cane_file_path = r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/RichardsonCane.csv"
gmu_file_path = r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/GMU.csv"
soho_file_path = r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/SOHO_LASCO.csv"
omniweb_file_path = r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/OMNIWeb.csv"

In [458]:
# Reading SOHO-LASCO dataset
soho_df = pd.read_csv(soho_file_path)
soho_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39285 entries, 0 to 39284
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          39285 non-null  object 
 1   Time          39285 non-null  object 
 2   CentralPA     39285 non-null  object 
 3   AngularWidth  39285 non-null  int64  
 4   LinearSpeed   39166 non-null  float64
 5   Accel         39285 non-null  object 
 6   MPA           39285 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 2.1+ MB


In [459]:
soho_df.head(200)

,Date,Time,CentralPA,AngularWidth,LinearSpeed,Accel,MPA
0,11/01/1996,00:14:36,267,18,499.0,-64.3*,272
1,13/01/1996,22:08:30,265,16,290.0,2.8*,266
2,15/01/1996,07:01:10,262,43,525.0,-31.1,272
3,22/01/1996,03:11:01,105,37,267.0,-126.3*,103
4,26/01/1996,09:16:19,90,27,262.0,1.9*,90
...,...,...,...,...,...,...,...
195,20/06/1996,13:53:43,73,16,220.0,0.8*,82
196,20/06/1996,17:49:03,105,38,80.0,6.6*,103
197,20/06/1996,19:18:00,1,6,91.0,-4.3*,360
198,21/06/1996,02:07:30,260,20,154.0,2.2*,265


After inspecting the dataset, we will do the following:
- Convert all missing values labelled as "------" and "NaN" to `None`.
- Convert Central PA values labelled as "Halo" to 360.
- Reformat the Acceleration column by removing asterisks.
- Convert all columns to numeric.
- Remove CME data corresponding to an angular width below 90 degrees as it is known that these are not likely to be geoeffective.
- Replace labelled columns "Date" and "Time" with a single "Datetime" column.

In [460]:
# Step 1: Replace all missing values ('------' and 'NaN') with None
soho_df.replace(['------', 'NaN'], None, inplace=True)

# Step 2: Convert Angular Width values labelled as "Halo" to 360
soho_df['CentralPA'] = soho_df['CentralPA'].replace('Halo', 360)

# Step 3: Remove asterisks from the Acceleration column
soho_df['Accel'] = soho_df['Accel'].astype(str).str.replace('*', '', regex=False)

# Step 4: Convert all columns to numeric, except the first two (Date and Time)
cols_to_convert = soho_df.columns[2:]  # Keep first two columns (Date and Time) as object
soho_df[cols_to_convert] = soho_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Step 5: Remove rows where Central PA is below 90 degrees
soho_df = soho_df[soho_df['AngularWidth'] >= 90]

# Step 6: Combine first two columns
soho_df['Datetime'] = pd.to_datetime(soho_df['Date'] + ' ' + soho_df['Time'], dayfirst=True) # Creating Datetime column
soho_df = soho_df.drop(soho_df.columns[[0, 1]], axis=1) # Drop the first two columns by index
last_column = soho_df.pop(soho_df.columns[-1])  # Pop the last column
soho_df.insert(0, last_column.name, last_column)  # Insert it at the front

soho_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5873 entries, 12 to 39282
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Datetime      5873 non-null   datetime64[ns]
 1   CentralPA     5873 non-null   int64         
 2   AngularWidth  5873 non-null   int64         
 3   LinearSpeed   5867 non-null   float64       
 4   Accel         5851 non-null   float64       
 5   MPA           5873 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 321.2 KB


In [461]:
soho_df.head(200) 

,Datetime,CentralPA,AngularWidth,LinearSpeed,Accel,MPA
12,1996-02-02 23:00:47,180,119,80.0,1.8,164
83,1996-04-29 14:38:48,360,360,65.0,NaN,149
85,1996-05-01 08:41:46,94,95,314.0,0.7,70
188,1996-06-18 17:28:50,84,95,64.0,-0.4,79
285,1996-07-20 09:28:16,31,175,246.0,9.4,34
...,...,...,...,...,...,...
2561,1998-11-07 20:54:05,321,96,750.0,23.7,314
2565,1998-11-08 11:54:05,264,196,559.0,6.2,214
2569,1998-11-09 01:54:05,16,94,144.0,0.7,27
2573,1998-11-09 18:17:55,330,190,325.0,2.6,338


#### OMNIWeb Plus Dataset
To obtain the final list of features, we must concatenate the SOHO-LASCO Catalogue with the OMNIWeb Plus Dataset. This dataset contains 12 features associated with the solar wind:

- The X-component of the magnetic field in nT.
- The Y-component of the magnetic field in nT.
- The Z-component of the magnetic field in nT.
- Plasma Temperature in Kelvin.
- Solar Proton Density n/cc.
- Flow Speed in km/s.
- Longitude Angle in degrees.
- Latitude Angle in degrees.
- Proton Density Ratio (unitless).
- Flow Pressure in nPa.
- Plasma Beta (unitless).
- Sunspot Number.

In [462]:
# Loading in OMNIWeb Plus data
omniweb_df = pd.read_csv(omniweb_file_path)
omniweb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261168 entries, 0 to 261167
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Datetime           261168 non-null  object 
 1   BX                 261168 non-null  float64
 2   BY                 261168 non-null  float64
 3   BZ                 261168 non-null  float64
 4   Plasma_Temp        261168 non-null  int64  
 5   Proton_Density     261168 non-null  float64
 6   Plasma_Speed       261168 non-null  int64  
 7   Plasma_Long_Angle  261168 non-null  float64
 8   Plasma_Lat_Angle   261168 non-null  float64
 9   Alpha_Prot_Ratio   261168 non-null  float64
 10  Flow_Pressure      261168 non-null  float64
 11  Plasma_Beta        261168 non-null  float64
 12  Sunspot_No         261168 non-null  int64  
dtypes: float64(9), int64(3), object(1)
memory usage: 25.9+ MB


In [463]:
omniweb_df.head()

,Datetime,BX,BY,BZ,Plasma_Temp,Proton_Density,Plasma_Speed,Plasma_Long_Angle,Plasma_Lat_Angle,Alpha_Prot_Ratio,Flow_Pressure,Plasma_Beta,Sunspot_No
0,01/01/1995 00:00,-0.6,3.2,-2.1,15816,16.3,315,-0.4,4.6,0.016,2.87,6.11,13
1,01/01/1995 01:00,-0.9,2.4,-1.3,15601,18.8,315,-0.1,3.1,0.013,3.28,12.51,13
2,01/01/1995 02:00,0.5,1.8,-0.4,20703,19.3,320,0.3,2.1,0.012,3.46,11.69,13
3,01/01/1995 03:00,-3.2,2.5,0.3,25094,16.4,317,1.6,2.2,0.013,2.90,5.66,13
4,01/01/1995 04:00,-4.3,2.1,0.3,25604,14.5,313,1.7,2.2,0.015,2.51,4.03,13


After taking a look at the OMNIWeb Plus Data, we next to the following to the dataset:
- Reformat first column to match the SOHO-LASCO data.
- Match times by averaging the 6-hour window after CME ejection.
- Concatenate both datasets to obtain final list of all 17 features.

In [464]:
# Step 1: Reformat Datetime column
omniweb_df['Datetime'] = pd.to_datetime(omniweb_df['Datetime'] + ':00', dayfirst=True)
omniweb_df.head(500)

,Datetime,BX,BY,BZ,Plasma_Temp,Proton_Density,Plasma_Speed,Plasma_Long_Angle,Plasma_Lat_Angle,Alpha_Prot_Ratio,Flow_Pressure,Plasma_Beta,Sunspot_No
0,1995-01-01 00:00:00,-0.6,3.2,-2.1,15816,16.3,315,-0.4,4.6,0.016,2.87,6.11,13
1,1995-01-01 01:00:00,-0.9,2.4,-1.3,15601,18.8,315,-0.1,3.1,0.013,3.28,12.51,13
2,1995-01-01 02:00:00,0.5,1.8,-0.4,20703,19.3,320,0.3,2.1,0.012,3.46,11.69,13
3,1995-01-01 03:00:00,-3.2,2.5,0.3,25094,16.4,317,1.6,2.2,0.013,2.90,5.66,13
4,1995-01-01 04:00:00,-4.3,2.1,0.3,25604,14.5,313,1.7,2.2,0.015,2.51,4.03,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1995-01-21 15:00:00,-3.6,1.2,-3.2,20014,11.3,348,-1.0,-0.3,0.025,2.51,2.68,74
496,1995-01-21 16:00:00,-3.1,2.6,1.2,31369,10.3,352,-1.0,-1.8,0.030,2.39,2.85,74
497,1995-01-21 17:00:00,-3.7,3.3,3.4,42588,10.5,356,-1.5,-2.3,0.039,2.57,1.77,74
498,1995-01-21 18:00:00,-3.7,4.2,0.7,46710,15.2,364,-3.2,-1.0,0.037,3.86,1.97,74


In [465]:
omniweb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261168 entries, 0 to 261167
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Datetime           261168 non-null  datetime64[ns]
 1   BX                 261168 non-null  float64       
 2   BY                 261168 non-null  float64       
 3   BZ                 261168 non-null  float64       
 4   Plasma_Temp        261168 non-null  int64         
 5   Proton_Density     261168 non-null  float64       
 6   Plasma_Speed       261168 non-null  int64         
 7   Plasma_Long_Angle  261168 non-null  float64       
 8   Plasma_Lat_Angle   261168 non-null  float64       
 9   Alpha_Prot_Ratio   261168 non-null  float64       
 10  Flow_Pressure      261168 non-null  float64       
 11  Plasma_Beta        261168 non-null  float64       
 12  Sunspot_No         261168 non-null  int64         
dtypes: datetime64[ns](1), float64(9), int64(3)
m

In [466]:
replace_values = [9999, 99, 999, 999.9, 9999999., 9999., 99.99, 9.999, 999.99, 999]
omniweb_df.replace(replace_values, None, inplace=True)

In [467]:
# Define a function to get the 6-hour averaged data after each CME
def get_solar_wind_average(cme_time, omniweb_data, window_hours=6):
    # Get the end time for the 6-hour window
    end_time = cme_time + pd.Timedelta(hours=window_hours)
    
    # Filter OMNIWeb data for the 6-hour window
    filtered_omniweb = omniweb_data[(omniweb_data['Datetime'] >= cme_time) & (omniweb_data['Datetime'] <= end_time)]
    
    # Calculate the average of all numerical columns in this window
    return filtered_omniweb.mean()

# Apply this function to each CME in the Cane dataset
averaged_solar_wind = soho_df['Datetime'].apply(get_solar_wind_average, omniweb_data=omniweb_df)

# Combine the averaged solar wind features with the original Cane dataset
combined_df = pd.concat([soho_df, averaged_solar_wind], axis=1)
combined_df.columns.values[6] = 'Datetime_2' # Renaming duplicated column

features_df  = combined_df.drop('Datetime_2', axis=1) # Removing duplicated Datetime column
features_df.head(200)


,Datetime,CentralPA,AngularWidth,LinearSpeed,Accel,MPA,BX,BY,BZ,Plasma_Temp,Proton_Density,Plasma_Speed,Plasma_Long_Angle,Plasma_Lat_Angle,Alpha_Prot_Ratio,Flow_Pressure,Plasma_Beta,Sunspot_No
12,1996-02-02 23:00:47,180,119,80.0,1.8,164,4.150000,-0.116667,3.333333e-02,75709.500000,3.466667,493.666667,1.466667,-1.000000,0.016667,1.506667,1.785000,12.000000
83,1996-04-29 14:38:48,360,360,65.0,NaN,149,-0.583333,0.300000,7.833333e-01,25115.666667,11.800000,366.500000,-0.083333,-0.066667,0.012500,2.780000,20.178333,0.000000
85,1996-05-01 08:41:46,94,95,314.0,0.7,70,3.083333,-0.366667,-1.850372e-17,49811.000000,6.900000,406.166667,2.850000,-1.916667,0.028667,2.116667,4.000000,0.000000
188,1996-06-18 17:28:50,84,95,64.0,-0.4,79,0.933333,-4.950000,-1.900000e+00,60472.500000,9.083333,388.000000,-1.900000,-2.700000,0.031667,2.596667,2.695000,14.000000
285,1996-07-20 09:28:16,31,175,246.0,9.4,34,0.533333,3.500000,-1.283333e+00,102147.166667,7.866667,422.500000,1.100000,-1.450000,0.019000,2.518333,3.020000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2561,1998-11-07 20:54:05,321,96,750.0,23.7,314,4.466667,10.183333,-5.783333e+00,36940.333333,7.083333,471.833333,-1.200000,-1.633333,0.022167,2.940000,0.351667,128.000000
2565,1998-11-08 11:54:05,264,196,559.0,6.2,214,-6.150000,-8.816667,2.028333e+01,111616.833333,18.650000,540.500000,2.600000,-1.933333,0.081167,11.976667,0.283333,121.000000
2569,1998-11-09 01:54:05,16,94,144.0,0.7,27,6.916667,12.350000,-5.350000e+00,32074.833333,8.333333,484.666667,-2.716667,3.200000,0.068667,4.171667,0.173333,93.000000
2573,1998-11-09 18:17:55,330,190,325.0,2.6,338,1.383333,-8.666667,-6.666667e+00,14778.500000,3.283333,422.166667,-0.916667,3.683333,0.072400,1.256667,0.128333,92.333333


In [468]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5873 entries, 12 to 39282
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Datetime           5873 non-null   datetime64[ns]
 1   CentralPA          5873 non-null   int64         
 2   AngularWidth       5873 non-null   int64         
 3   LinearSpeed        5867 non-null   float64       
 4   Accel              5851 non-null   float64       
 5   MPA                5873 non-null   int64         
 6   BX                 5786 non-null   float64       
 7   BY                 5786 non-null   float64       
 8   BZ                 5786 non-null   float64       
 9   Plasma_Temp        5738 non-null   float64       
 10  Proton_Density     5725 non-null   float64       
 11  Plasma_Speed       5843 non-null   float64       
 12  Plasma_Long_Angle  5843 non-null   float64       
 13  Plasma_Lat_Angle   5843 non-null   float64       
 14  Alpha_Prot_

In [469]:
features_df.to_csv(r'/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/features.csv', index=False)

#### Target Geoeffective CMEs and Transit Times


In [470]:
cane_df = pd.read_csv(cane_file_path)
cane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   LASCO CME Y/M/D (UT)    558 non-null    object
 1   Disturbance Y/M/D (UT)  558 non-null    object
 2   Dst (nT)                558 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 13.2+ KB


In [471]:
cane_df.head(100)

,LASCO CME Y/M/D (UT),Disturbance Y/M/D (UT),Dst (nT)
0,1996/05/27 1500,1996/05/27 1500,-33
1,1996/07/01 1800,1996/07/01 1320,-20
2,1996/08/07 1200,1996/08/07 0600,-23
3,1996/12/19 1630,1996/12/23 1600,-18
4,1997/01/06 1510,1997/01/10 0104,-78
...,...,...,...
95,2000/01/18 1754,2000/01/22 0023,-97
96,2000/02/08 0930,2000/02/11 0258,-25
97,2000/02/10 0230,2000/02/11 2352,-133
98,2000/02/12 0431,2000/02/14 0731,-67


In [472]:
cane_df['LASCO CME Y/M/D (UT)'] = pd.to_datetime(cane_df['LASCO CME Y/M/D (UT)'], format='mixed')
cane_df['Disturbance Y/M/D (UT)'] = pd.to_datetime(cane_df['Disturbance Y/M/D (UT)'], format='mixed')
cane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   LASCO CME Y/M/D (UT)    558 non-null    datetime64[ns]
 1   Disturbance Y/M/D (UT)  558 non-null    datetime64[ns]
 2   Dst (nT)                558 non-null    int64         
dtypes: datetime64[ns](2), int64(1)
memory usage: 13.2 KB


In [473]:
cane_df.head()

,LASCO CME Y/M/D (UT),Disturbance Y/M/D (UT),Dst (nT)
0,1996-05-27 15:00:00,1996-05-27 15:00:00,-33
1,1996-07-01 18:00:00,1996-07-01 13:20:00,-20
2,1996-08-07 12:00:00,1996-08-07 06:00:00,-23
3,1996-12-19 16:30:00,1996-12-23 16:00:00,-18
4,1997-01-06 15:10:00,1997-01-10 01:04:00,-78


In [474]:
cane_df = cane_df[cane_df['Dst (nT)'] <= -30]

In [475]:
cane_df.head(500)

,LASCO CME Y/M/D (UT),Disturbance Y/M/D (UT),Dst (nT)
0,1996-05-27 15:00:00,1996-05-27 15:00:00,-33
4,1997-01-06 15:10:00,1997-01-10 01:04:00,-78
5,1997-02-07 00:30:00,1997-02-09 13:21:00,-68
6,1997-04-07 14:27:00,1997-04-10 17:45:00,-82
7,1997-04-21 10:00:00,1997-04-21 06:00:00,-107
...,...,...,...
552,2023-09-29 10:00:00,2023-09-28 21:31:00,-47
554,2023-11-09 12:23:00,2023-11-12 06:13:00,-42
555,2023-11-28 20:24:00,2023-12-01 09:26:00,-107
556,2023-12-12 00:00:00,2023-12-15 11:48:00,-33


In [476]:
cane_df['TransitTime'] = cane_df['Disturbance Y/M/D (UT)'] - cane_df['LASCO CME Y/M/D (UT)']

In [477]:
cane_df.head()

,LASCO CME Y/M/D (UT),Disturbance Y/M/D (UT),Dst (nT),TransitTime
0,1996-05-27 15:00:00,1996-05-27 15:00:00,-33,0 days 00:00:00
4,1997-01-06 15:10:00,1997-01-10 01:04:00,-78,3 days 09:54:00
5,1997-02-07 00:30:00,1997-02-09 13:21:00,-68,2 days 12:51:00
6,1997-04-07 14:27:00,1997-04-10 17:45:00,-82,3 days 03:18:00
7,1997-04-21 10:00:00,1997-04-21 06:00:00,-107,-1 days +20:00:00


In [478]:
cane_df = cane_df[cane_df['TransitTime'].dt.total_seconds() > 0]
cane_df['TransitTime'] = cane_df['TransitTime'].dt.total_seconds() / 3600

In [479]:
cane_df['Geoeffective'] = 1
cane_df = cane_df.drop(cane_df.columns[[1, 2]], axis=1)
cane_df.head()

,LASCO CME Y/M/D (UT),TransitTime,Geoeffective
4,1997-01-06 15:10:00,81.900000,1
5,1997-02-07 00:30:00,60.850000,1
6,1997-04-07 14:27:00,75.300000,1
8,1997-05-12 05:30:00,68.483333,1
9,1997-05-21 21:00:00,108.950000,1


In [480]:
cane_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 238 entries, 4 to 557
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   LASCO CME Y/M/D (UT)  238 non-null    datetime64[ns]
 1   TransitTime           238 non-null    float64       
 2   Geoeffective          238 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 7.4 KB


In [481]:
gmu_df = pd.read_csv(gmu_file_path)

In [482]:
gmu_df.head()


,CME in LASCO,Transit time,Geoeffective
0,11/16/2007 08:50:00,87:10:00,1
1,12/12/2008 09:06:00,112:54:00,1
2,01/21/2009 21:08:00,103:52:00,1
3,01/31/2009 01:54:00,89:06:00,1
4,09/25/2009 08:06:00,117:54:00,1


In [483]:
gmu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CME in LASCO  72 non-null     object
 1   Transit time  72 non-null     object
 2   Geoeffective  72 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.8+ KB


In [484]:

gmu_df['Transit time'] = pd.to_timedelta(gmu_df['Transit time'])
gmu_df['Transit time'] = gmu_df['Transit time'].dt.total_seconds() / 3600

In [485]:
gmu_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CME in LASCO  72 non-null     object 
 1   Transit time  72 non-null     float64
 2   Geoeffective  72 non-null     int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.8+ KB


In [486]:
gmu_df.head(100)

,CME in LASCO,Transit time,Geoeffective
0,11/16/2007 08:50:00,87.166667,1
1,12/12/2008 09:06:00,112.900000,1
2,01/21/2009 21:08:00,103.866667,1
3,01/31/2009 01:54:00,89.100000,1
4,09/25/2009 08:06:00,117.900000,1
...,...,...,...
67,11/04/2015 14:48:00,13.350000,1
68,12/28/2015 12:12:05,15.766667,1
69,07/17/2016 12:12:05,46.000000,1
70,10/09/2016 02:36:00,49.600000,1


In [487]:
#save CSV
gmu_df.to_csv(r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/cleaned_gmu.csv", index=False)

In [488]:
cane_df.to_csv(r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/cleaned_cane.csv", index=False)

In [489]:
from datetime import timedelta


# Convert the datetime columns to a consistent format for merging
features_df['Datetime'] = pd.to_datetime(features_df['Datetime'])
cane_df['LASCO CME Y/M/D (UT)'] = pd.to_datetime(cane_df['LASCO CME Y/M/D (UT)'])
gmu_df['CME in LASCO'] = pd.to_datetime(gmu_df['CME in LASCO'])

# Rename columns for consistency
cane_df.rename(columns={'LASCO CME Y/M/D (UT)': 'Datetime'}, inplace=True)
gmu_df.rename(columns={'CME in LASCO': 'Datetime'}, inplace=True)

# Merge the cleaned_cane and cleaned_gmu datasets together
geoeffective_df = pd.concat([cane_df, gmu_df])

# Remove duplicate CMEs from the combined geoeffective dataframe based on the datetime
geoeffective_df = geoeffective_df.drop_duplicates(subset='Datetime')

# Set an epsilon time window (e.g., +/- 6 hours)
epsilon = timedelta(hours=0.5)

# Create new columns for Geoeffective and TransitTime with default values
merged_df = features_df.copy()
merged_df['Geoeffective'] = 0
merged_df['TransitTime'] = None

# Iterate over each geoeffective event and match with the features within the epsilon window
for _, row in geoeffective_df.iterrows():
    cme_datetime = row['Datetime']
    
    # Find matches within the epsilon window
    mask = (merged_df['Datetime'] >= cme_datetime - epsilon) & (merged_df['Datetime'] <= cme_datetime + epsilon)
    merged_df.loc[mask, 'Geoeffective'] = 1
    merged_df.loc[mask, 'TransitTime'] = row['TransitTime']

# Optionally, display the first few rows of the updated merged dataframe
merged_df.head()

,Datetime,CentralPA,AngularWidth,LinearSpeed,Accel,MPA,BX,BY,BZ,Plasma_Temp,Proton_Density,Plasma_Speed,Plasma_Long_Angle,Plasma_Lat_Angle,Alpha_Prot_Ratio,Flow_Pressure,Plasma_Beta,Sunspot_No,Geoeffective,TransitTime
12,1996-02-02 23:00:47,180,119,80.0,1.8,164,4.150000,-0.116667,3.333333e-02,75709.500000,3.466667,493.666667,1.466667,-1.000000,0.016667,1.506667,1.785000,12.0,0,None
83,1996-04-29 14:38:48,360,360,65.0,NaN,149,-0.583333,0.300000,7.833333e-01,25115.666667,11.800000,366.500000,-0.083333,-0.066667,0.012500,2.780000,20.178333,0.0,0,None
85,1996-05-01 08:41:46,94,95,314.0,0.7,70,3.083333,-0.366667,-1.850372e-17,49811.000000,6.900000,406.166667,2.850000,-1.916667,0.028667,2.116667,4.000000,0.0,0,None
188,1996-06-18 17:28:50,84,95,64.0,-0.4,79,0.933333,-4.950000,-1.900000e+00,60472.500000,9.083333,388.000000,-1.900000,-2.700000,0.031667,2.596667,2.695000,14.0,0,None
285,1996-07-20 09:28:16,31,175,246.0,9.4,34,0.533333,3.500000,-1.283333e+00,102147.166667,7.866667,422.500000,1.100000,-1.450000,0.019000,2.518333,3.020000,0.0,0,None


In [490]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5873 entries, 12 to 39282
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Datetime           5873 non-null   datetime64[ns]
 1   CentralPA          5873 non-null   int64         
 2   AngularWidth       5873 non-null   int64         
 3   LinearSpeed        5867 non-null   float64       
 4   Accel              5851 non-null   float64       
 5   MPA                5873 non-null   int64         
 6   BX                 5786 non-null   float64       
 7   BY                 5786 non-null   float64       
 8   BZ                 5786 non-null   float64       
 9   Plasma_Temp        5738 non-null   float64       
 10  Proton_Density     5725 non-null   float64       
 11  Plasma_Speed       5843 non-null   float64       
 12  Plasma_Long_Angle  5843 non-null   float64       
 13  Plasma_Lat_Angle   5843 non-null   float64       
 14  Alpha_Prot_

In [491]:
geoeffective_count_merged = merged_df['Geoeffective'].sum()

237

In [493]:
merged_df.to_csv(r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/final_dataset.csv", index=False)